In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
import nltk
import numpy as np

c:\Users\chang\anaconda3\envs\Semantic_SBERT2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate(gcs): #所以gcs要是string

    
    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    for t in range(0,480):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))

    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    global SBERT 
    SBERT= []

    def get_cosine_similarity(feature_vec_1, feature_vec_2):    
        return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
    # Two lists of sentences
    for i in range(0,len(all_sentences001)-1):
        #Compute embedding for both lists
        embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
        embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)

        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        # convert from PyTorch tensor to numpy array
        SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
    ######
    
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    for t in range(0,480):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global pair_counts
    pair_counts = []
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        wu_pulmer_sum = 0
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):
                        for u in range(0, syns2_num):
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                            timeframe_pairs_count += 1
                else:
                    continue
        if timeframe_pairs_count != 0:
            all_sentences_wu_sum_divided.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
        else: 
            all_sentences_wu_sum_divided.append(wu_pulmer_sum)
        all_sentences_wu_sum.append(wu_pulmer_sum)
        pair_counts.append(timeframe_pairs_count)
        #print(all_sentences_wu_sum)
        global wu_difference
        wu_difference = []
        for x in range(len(all_sentences_wu_sum)-1):
            wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])

        from sklearn import preprocessing
    
    """ # for normalization
    wu_difference = np.array(wu_difference)
    SBERT = np.array(SBERT)
    #normalized_SBERT = preprocessing.normalize([SBERT])
    #normalized_wu_difference = preprocessing.normalize([wu_difference])


    min_max_scaler = preprocessing.MinMaxScaler()
    SBERT = np.reshape(SBERT,(479,1))
    wu_difference = np.reshape(wu_difference,(479,1))
    global normalized_SBERT
    global normalized_wu_difference
    normalized_SBERT  = min_max_scaler.fit_transform(SBERT)
    normalized_wu_difference  = min_max_scaler.fit_transform(wu_difference)
    """

參數表:<br>
all_sentences001: 所有的句子整合(沒有刪掉stop words)<br>
SBERT: 具有全部SBERT similarity的list<br>
all_sentences_wu_sum: 每句話的wu-pulmer similarity算出來的平均的list<br>
wu_difference: 兩句兩句減掉wu similarity的list<br>
normalized_SBERT: 就是表準化過後的SBERT list<br>
normalized_wu_difference: 標準化過後的wu_pulmer difference<br>
這邊注意wu-pulmer的different based應該不能這樣算

In [3]:
'''
i = 6
calculate(f"data/Text/00{i}(done).csv")
results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
results_csv.to_csv(f'data/Text/00{i}(results).csv')
'''

'\ni = 6\ncalculate(f"data/Text/00{i}(done).csv")\nresults_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\nresults_csv.to_csv(f\'data/Text/00{i}(results).csv\')\n'

In [4]:
for i in range(7,24):
    if i<10:
        try:
            calculate(f"data/Text/00{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/00{i}(results).csv')
        except:
            print("error:", i)
            continue
    else:
        try:
            calculate(f"data/Text/0{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/0{i}(results).csv')
        except:
            print("error:", i)
            continue

error: 9
error: 10
error: 12
error: 16
error: 18
error: 22


In [5]:
print(SBERT[0:20])

[0.544697, 0.8420632, 0.97001255, 0.8372743, 0.831531, 0.95109665, 0.887207, 0.81038785, 0.8672111, 0.98093855, 0.7542483, 1.0, 0.9930301, 0.665328, 0.99638027, 0.8780438, 0.8462553, 0.5799331, 0.98352784, 0.888752]


In [6]:
results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
print(results_csv.tail(5))

     SBERT  Wu-Pulmer  Wu-Pulmer_divided  pair_counts
474    1.0        0.0                0.0            0
475    1.0        0.0                0.0            0
476    1.0        0.0                0.0            0
477    1.0        0.0                0.0            0
478    1.0        0.0                0.0            0
